# Volatility Measures Comparison

This notebook compares different ways to measure Bitcoin volatility and their sensitivity to market events.

## Volatility Measures Tested:
1. **Absolute Returns (Abs_Return)** - Simple daily price change magnitude
2. **30-day Rolling Volatility** - Standard deviation of returns over 30 days
3. **GARCH Volatility** - Model-based conditional volatility (if implemented)

## Objectives:
- Compare sensitivity of different volatility measures to events
- Identify which measure best captures event impacts
- Analyze the relationship between measures


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Load data
project_root = Path('..')
btc_data = pd.read_csv(project_root / 'data' / 'raw' / 'bitcoin_prices.csv', 
                       parse_dates=['Date'], index_col='Date')
events_data = pd.read_csv(project_root / 'data' / 'processed' / 'market_events.csv', 
                          parse_dates=['date'])

print("Data loaded successfully!")
print(f"Bitcoin data: {btc_data.shape}")
print(f"Events data: {events_data.shape}")

# Check available volatility measures
print(f"\nAvailable volatility measures in data:")
vol_measures = ['Abs_Return', 'Volatility_30d', 'Daily_Return']
for measure in vol_measures:
    if measure in btc_data.columns:
        print(f"  ✓ {measure}")
    else:
        print(f"  ✗ {measure}")


In [ ]:
# Create additional volatility measures for comparison
print("Creating additional volatility measures...")

# 1. 7-day rolling volatility
btc_data['Volatility_7d'] = btc_data['Daily_Return'].rolling(window=7).std()

# 2. 14-day rolling volatility  
btc_data['Volatility_14d'] = btc_data['Daily_Return'].rolling(window=14).std()

# 3. Realized volatility (using absolute returns over different windows)
btc_data['Realized_Vol_7d'] = btc_data['Abs_Return'].rolling(window=7).mean()
btc_data['Realized_Vol_14d'] = btc_data['Abs_Return'].rolling(window=14).mean()

# 4. High-Low volatility proxy
btc_data['HL_Volatility'] = (btc_data['High'] - btc_data['Low']) / btc_data['Close']

print("Additional volatility measures created:")
new_measures = ['Volatility_7d', 'Volatility_14d', 'Realized_Vol_7d', 'Realized_Vol_14d', 'HL_Volatility']
for measure in new_measures:
    print(f"  ✓ {measure}")

# Show correlation matrix of all volatility measures
vol_columns = ['Abs_Return', 'Volatility_30d', 'Volatility_7d', 'Volatility_14d', 
               'Realized_Vol_7d', 'Realized_Vol_14d', 'HL_Volatility']
vol_data = btc_data[vol_columns].dropna()

print(f"\nCorrelation matrix of volatility measures:")
correlation_matrix = vol_data.corr()
print(correlation_matrix.round(3))


In [ ]:
# Visualize volatility measures comparison
fig, axes = plt.subplots(3, 2, figsize=(15, 12))

# Plot all volatility measures over time
vol_measures_to_plot = ['Abs_Return', 'Volatility_7d', 'Volatility_14d', 'Volatility_30d', 'HL_Volatility']
colors = ['blue', 'green', 'orange', 'red', 'purple']

for i, (measure, color) in enumerate(zip(vol_measures_to_plot, colors)):
    row = i // 2
    col = i % 2
    if row < 3 and col < 2:
        axes[row, col].plot(btc_data.index, btc_data[measure], color=color, alpha=0.7, linewidth=1)
        axes[row, col].set_title(f'{measure}', fontweight='bold')
        axes[row, col].set_ylabel('Volatility')
        axes[row, col].grid(True, alpha=0.3)

# Remove empty subplot
if len(vol_measures_to_plot) < 6:
    axes[2, 1].remove()

plt.tight_layout()
plt.show()

# Correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.3f', cbar_kws={'label': 'Correlation'})
plt.title('Volatility Measures Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# Test event sensitivity for different volatility measures
from datetime import timedelta

def test_volatility_sensitivity(btc_data, events_data, vol_measure, window_days=10):
    """Test how sensitive a volatility measure is to events"""
    results = []
    
    for idx, event in events_data.iterrows():
        event_date = event['date']
        
        # Define before and after periods
        before_start = event_date - timedelta(days=window_days)
        before_end = event_date - timedelta(days=1)
        after_start = event_date + timedelta(days=1)
        after_end = event_date + timedelta(days=window_days)
        
        # Get data
        before_data = btc_data[(btc_data.index >= before_start) & 
                              (btc_data.index <= before_end)][vol_measure].dropna()
        after_data = btc_data[(btc_data.index >= after_start) & 
                             (btc_data.index <= after_end)][vol_measure].dropna()
        
        # Calculate change and significance
        if len(before_data) > 3 and len(after_data) > 3:
            before_mean = before_data.mean()
            after_mean = after_data.mean()
            change = after_mean - before_mean
            change_pct = (change / before_mean) * 100 if before_mean != 0 else 0
            
            # T-test
            t_stat, p_value = stats.ttest_ind(before_data, after_data)
            significant = p_value < 0.05
        else:
            before_mean = after_mean = change = change_pct = t_stat = p_value = np.nan
            significant = False
        
        results.append({
            'event_id': event['event_id'],
            'event': event['event'],
            'severity': event['severity'],
            'before_mean': before_mean,
            'after_mean': after_mean,
            'change': change,
            'change_pct': change_pct,
            't_statistic': t_stat,
            'p_value': p_value,
            'significant': significant
        })
    
    return pd.DataFrame(results)

# Test all volatility measures
print("Testing event sensitivity for different volatility measures...")
vol_measures_to_test = ['Abs_Return', 'Volatility_7d', 'Volatility_14d', 'Volatility_30d', 'HL_Volatility']
sensitivity_results = {}

for measure in vol_measures_to_test:
    print(f"\nTesting {measure}...")
    results = test_volatility_sensitivity(btc_data, events_data, measure)
    sensitivity_results[measure] = results
    
    significant_count = results['significant'].sum()
    mean_change = results['change'].mean()
    print(f"  Significant events: {significant_count}/{len(results)} ({significant_count/len(results)*100:.1f}%)")
    print(f"  Mean change: {mean_change:.4f}")

# Summary comparison
print(f"\n=== VOLATILITY MEASURES SENSITIVITY SUMMARY ===")
summary_data = []
for measure, results in sensitivity_results.items():
    summary_data.append({
        'Measure': measure,
        'Significant_Events': results['significant'].sum(),
        'Significance_Rate': results['significant'].sum() / len(results) * 100,
        'Mean_Change': results['change'].mean(),
        'Mean_Change_Pct': results['change_pct'].mean()
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.round(3))
